In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import pyautogui

def select_match(driver, match_id):
    # Function to select a specific match
    # Implement your logic to select a match based on match_id
    pass


def scrape_matchinfo(driver):
    # Function to scrape match information
    match_info = {}
    # Implement your scraping logic here
    return match_info

def scrape_homepage(driver,sport,quarter_type,bet_type,date):
    # Function to scrape all text on the homepage
    rows_data = []
    columns = {
        '1': 'match_info',
        '2': 'score',
        '3': 'col_3',
        '4': 'col_4'
    }
    column_titles = {}  # To store the column titles found

    try:
        # Wait until the page loads and an element we are interested in is present
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ag-pinned-left-cols-container')))
        
        # Get the container with all the rows in the pinned left columns
        left_container = driver.find_element(By.CSS_SELECTOR, 'div.ag-pinned-left-cols-container')

        # Iterate through each row in the pinned left columns
        row_index = 0
        while True:
            try:
                # Locate each row by row index
                left_row = left_container.find_element(By.CSS_SELECTOR, f'div[row-index="{row_index}"]')
                # Extract the text or any specific data from the row, including column index
                row_data = {'row_index': row_index}
                cells = left_row.find_elements(By.CSS_SELECTOR, 'div[aria-colindex]')
                for cell in cells:
                    col_index = cell.get_attribute('aria-colindex')
                    column_name = columns.get(col_index, f'col_{col_index}')
                    row_data[column_name] = cell.text
                rows_data.append(row_data)
                row_index += 1
            except Exception as e:
                print(f"Left container row parsing ended at index {row_index} with error: {e}")
                break

        # Now, get the container with all the rows in the center columns
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ag-center-cols-container')))
        center_container = driver.find_element(By.CSS_SELECTOR, 'div.ag-center-cols-container')

        # Iterate through each row in the center columns and merge with existing data
        for row in rows_data:
            try:
                # Locate each row by row index
                center_row = center_container.find_element(By.CSS_SELECTOR, f'div[row-index="{row["row_index"]}"]')
                # Extract the text or any specific data from the row, including column index
                cells = center_row.find_elements(By.CSS_SELECTOR, 'div[aria-colindex]')
                for cell in cells:
                    col_index = cell.get_attribute('aria-colindex')
                    column_name = columns.get(col_index, f'col_{col_index}')
                    row[column_name] = cell.text
            except Exception as e:
                print(f"Center container row parsing error for row index {row['row_index']}: {e}")
                continue

        # Get the column titles from the header starting at aria-colindex 5
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ag-header-row.ag-header-row-column')))
        header_rows = driver.find_elements(By.CSS_SELECTOR, 'div.ag-header-row.ag-header-row-column')
        
        #print("Found header rows:", len(header_rows))  # Debugging line
        for header_row in header_rows:
            headers = header_row.find_elements(By.CSS_SELECTOR, 'div[aria-colindex]')
            for header in headers:
                col_index = header.get_attribute('aria-colindex')
                if int(col_index) >= 5:
                    try:
                        # Inspect the header element
                        print(f"Inspecting header with aria-colindex {col_index}: {header.get_attribute('outerHTML')}")
                        title_element = header.find_element(By.CSS_SELECTOR, 'div[title]')
                        column_titles[col_index] = title_element.get_attribute('title')
                    except Exception as e:
                        print(f"Error finding title for col_index {col_index}: {e}")
                        column_titles[col_index] = f'col_{col_index}'
                    # Debug: Print the aria-colindex and the corresponding title
                    print(f'aria-colindex: {col_index}, title: {column_titles[col_index]}')

        # Rename the columns in the DataFrame
        df = pd.DataFrame(rows_data)
        df.rename(columns={f'col_{k}': v for k, v in column_titles.items()}, inplace=True)
        
        # Save the DataFrame to a CSV file
        df.to_csv(f'unabated-database/{sport}/{quarter_type}/{bet_type}/pregame/pregame_{sport}_{quarter_type}_{bet_type}_{date}.csv', index=False)
        print(f"Data saved to pregame_{sport}_{quarter_type}_{bet_type}_{date}.csv")

    except Exception as e:
        print(f"An error occurred: {e}")

def print_currentpage(driver):
    # Function to scrape the current page into text and print it
    try:
        while True:
            # Wait until the page loads and an element we are interested in is present
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            
            # Get all text content from the body
            body_text = driver.find_element(By.TAG_NAME, "body").text
            
            # Print the text content of the current page
            print(body_text)
            
            # Wait for 10 seconds before the next iteration
            time.sleep(10)
    except Exception as e:
        print(f"An error occurred: {e}")

def scrape_pregame(driver):
    # Function to scrape pregame odds
    pregame_odds = {}
    # Implement your scraping logic here
    return pregame_odds

def scrape_live(driver,sport,quarter_type,bet_type,date):
    # Function to scrape live odds
    all_live_data = []  # To store all parsed DataFrames

    try:
        # Wait until the page loads and the element we are interested in is present
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ag-center-cols-container')))
        
        # Get the container with all the rows in the center columns
        center_container = driver.find_element(By.CSS_SELECTOR, 'div.ag-center-cols-container')

        row_index = 0
        while True:
            try:
                # Locate each row by row index
                row = center_container.find_element(By.CSS_SELECTOR, f'div[row-index="{row_index}"]')
                
                col_index = 5
                while True:
                    try:
                        # Locate each cell by col index
                        cell = row.find_element(By.CSS_SELECTOR, f'div[aria-colindex="{col_index}"]')
                        react_container = cell.find_element(By.CSS_SELECTOR, 'div.ag-react-container')
                        
                        # Hover over and click the cell
                        ActionChains(driver).move_to_element(react_container).click().perform()
                        
                        # Wait for the textbox to appear and parse the data
                        time.sleep(0.5)  # Adjust sleep time as needed
                        df = parse_live_lines(driver,col_index,row_index)
                        all_live_data.append(df)
                        
                        col_index += 1
                    except Exception as e:
                        print(f"End of columns for row index {row_index} with error: {e}")
                        break
                
                row_index += 1
            except Exception as e:
                print(f"End of rows reached with error: {e}")
                break

        # Concatenate all DataFrames
        final_df = pd.concat(all_live_data, ignore_index=True)
        
        # Save the final DataFrame to a CSV file
        final_df.to_csv(f'unabated-database/{sport}/{quarter_type}/{bet_type}/live/live_{sport}_{quarter_type}_{bet_type}_{date}.csv', index=False)
        print(f"Live lines data saved to live_{sport}_{quarter_type}_{bet_type}_{date}.csv")

    except Exception as e:
        print(f"An error occurred while scraping live odds: {e}")

def parse_live_lines(driver, maincol, mainrow):
    
    # Disable PyAutoGUI fail-safe feature
    pyautogui.FAILSAFE = False

    # Function to parse the live lines data
    live_data = []
    read_rows = set()  # To keep track of row indexes that have been read
    try:
        # Wait until the element we are interested in is present
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.jsx-619290666')))
        
        # Get the container with the class jsx-619290666
        try:
            jsx_container = driver.find_element(By.CSS_SELECTOR, 'div.jsx-619290666')
            print("Found jsx_container")
        except Exception as e:
            print(f"Data container not found: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if the container is not found
        
        # Click the button with id="buttonGameStatusLive"
        try:
            live_button = jsx_container.find_element(By.ID, 'buttonGameStatusLive')
            live_button.click()
            print("Clicked live_button")
            time.sleep(1)  # Wait for the data to load
        except Exception as e:
            print(f"Live button not found or not clickable: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if the button is not found or not clickable

        # Wait until the element we are interested in is present
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ag-center-cols-container')))
        
        # Get the container with all the rows in the center columns
        center_container = jsx_container.find_element(By.CSS_SELECTOR, 'div.ag-center-cols-container')
        print("Found center_container")

        tries = 0
        
        while True:
            try:
                # Locate all rows in the center container
                rows = center_container.find_elements(By.CSS_SELECTOR, 'div[role="row"]')
                for row in rows:
                    row_index = int(row.get_attribute('row-index'))
                    if row_index not in read_rows:  # Only process rows that haven't been read
                        try:
                            # Extract the data for each column
                            time_text = row.find_element(By.CSS_SELECTOR, 'div[aria-colindex="1"]').text
                            away_text = row.find_element(By.CSS_SELECTOR, 'div[aria-colindex="2"]').text
                            home_text = row.find_element(By.CSS_SELECTOR, 'div[aria-colindex="3"]').text
                            available = "Closed" if "line-through" in\
                                row.find_element(By.CSS_SELECTOR, 'div[aria-colindex="2"]').find_element(By.CSS_SELECTOR, 'span.pr-2').get_attribute("style") else "Open"
                            
                            
                            live_data.append({
                                "Time": time_text,
                                "Away": away_text,
                                "Home": home_text,
                                "Available": available,
                                "Row": f"row_{mainrow}",
                                "Column": f"col_{maincol}"
                            })
                            #print(f"Parsed row {row_index}: Time={time_text}, Away={away_text}, Home={home_text}, Available={available}, Column=col_{maincol}")
                            read_rows.add(row_index)  # Mark this row as read
                        except Exception as e:
                            print(f"Error parsing row {row_index}: {e}")
                            continue
                
                # Click at the bottom-right to load more rows
                #window_size = driver.get_window_size()
                pyautogui.click()
                print("Clicked at the bottom-right corner to load more rows")
                time.sleep(0.5)

                # Check if new rows have been loaded
                hasnewrow = False
                new_rows = center_container.find_elements(By.CSS_SELECTOR, 'div[role="row"]')
                for row in new_rows:
                    row_index = int(row.get_attribute('row-index'))
                    if row_index not in read_rows:
                        hasnewrow = True
                        
                tries += 1
                
                if (hasnewrow==False and tries > 0):
                    print("No new rows loaded, finishing up")
                    break  # Exit if no new rows were loaded

            except Exception as e:
                print(f"Scroll bar click failed with error: {e}")
                break

        # Click the button with class="close" to close the data table
        try:
            close_button = driver.find_element(By.CSS_SELECTOR, 'button.close')
            close_button.click()
            print("Clicked close_button")
        except Exception as e:
            print(f"Close button not found or not clickable: {e}")

        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(live_data)
        return df

    except Exception as e:
        print(f"An error occurred while parsing live lines: {e}")
        return pd.DataFrame()





In [2]:
import pandas as pd
import os

def merge_info(live_path='live.csv', pregame_path='pregame.csv', output_path='merged.csv'):
    print('MERGEINFO')
    try:
        # Check if the input files exist and are not empty
        if os.stat(live_path).st_size == 0 or os.stat(pregame_path).st_size == 0:
            raise ValueError("One of the input files is empty.")

        # Load the live_lines.csv and nba_matches.csv files into DataFrames
        live_df = pd.read_csv(live_path)
        pregame_df = pd.read_csv(pregame_path)

        # Strip whitespace from column names in nba_matches_df
        pregame_df.columns = pregame_df.columns.str.strip()

        # Remove double spaces and leading/trailing spaces from every cell in nba_matches_df
        pregame_df = pregame_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

        #print(pregame_df.columns)  # For debugging, print the cleaned column names
        #print(live_df.columns)  # For debugging, print the cleaned column names

        # Initialize new columns in live_lines_df for match_info and score
        live_df['match_info'] = ''
        live_df['score'] = ''
        live_df['pregame'] = ''
        live_df['book'] = ''

        # Iterate through each row in live_lines_df
        for index, row in live_df.iterrows():
            try:
                # Extract row and column info from live_lines_df
                row_str = row['Row']
                col_str = row['Column']

                # Extract the row_index and col_index
                row_index = int(row_str.split('_')[1])
                col_index = int(col_str.split('_')[1])

                # Find the corresponding row in nba_matches_df
                match_info = pregame_df.loc[pregame_df['row_index'] == row_index, 'match_info'].values[0]
                score = pregame_df.loc[pregame_df['row_index'] == row_index, 'score'].values[0]

                # Find the corresponding column (e.g., col_5 -> the correct bookmaker)
                col_name = pregame_df.columns[col_index]
                pregame = pregame_df.loc[pregame_df['row_index'] == row_index, col_name].values[0]
                book = pregame_df.columns[col_index]

                # Update the live_lines_df with the matched info
                live_df.at[index, 'match_info'] = match_info
                live_df.at[index, 'score'] = score
                live_df.at[index, 'pregame'] = pregame
                live_df.at[index, 'book'] = book
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                raise

        # Apply cleaning to live_lines_df as well to remove excess spaces
        live_df = live_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

        # Save the updated DataFrame to a new CSV file
        live_df.to_csv(output_path, index=False)
        print(f"Merged data saved to {output_path}")

    except Exception as e:
        print(f"An error occurred: {e}. Saving an empty output CSV.")
        
        # Define the columns that would have been in the merged CSV
        columns = ['match_info', 'score', 'pregame', 'book', 'Time', 'Away', 'Home', 'Available', 'Row', 'Column']
        
        # Create an empty DataFrame with these columns
        empty_df = pd.DataFrame(columns=columns)
        
        # Save the empty DataFrame to the output path
        empty_df.to_csv(output_path, index=False)
        print(f"Empty merged data saved to {output_path}")


In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import time

def change_date(driver, day):
    try:
        print(f"Attempting to change date to: {day}")

        # Locate and click the date picker input to display the calendar
        date_picker = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'react-datepicker__input-container'))
        )
        date_picker.click()

        # Wait for the calendar to be visible and interactable
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, 'react-datepicker'))
        )

        # Click on the specific day that is clickable (not disabled or outside the month)
        day_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'react-datepicker__day') and text()='{day}' and not(contains(@class, 'react-datepicker__day--outside-month')) and not(@aria-disabled='true')]"))
        )
        day_element.click()
        print(f"Successfully changed date to: {day}")

    except TimeoutException:
        print(f"TimeoutException: Failed to find or click the date {day}.")
    except StaleElementReferenceException:
        print(f"StaleElementReferenceException: The page structure might have changed for date {day}.")
    except Exception as e:
        print(f"An error occurred while changing the date to {day}: {e}")

def go_to_next_month(driver):
    # Click the button to go to the next month
    try:
        date_picker = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'react-datepicker__input-container'))
        )
        date_picker.click()
        
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.react-datepicker__navigation--next"))
        )
        next_button.click()
    except Exception as e:
        print(f"Error clicking next month: {e}")

def get_available_days(driver):
    # Fetch and return the available day numbers as text
    date_picker = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'react-datepicker__input-container'))
    )
    date_picker.click()

    available_days_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'react-datepicker__day') and not(contains(@class, 'react-datepicker__day--outside-month')) and not(@aria-disabled='true')]"))
    )
    
    available_days = [day.text.strip() for day in available_days_elements if day.text.strip().isdigit()]
    print(f"Found {len(available_days)} available days to scan: {available_days}")
    
    date_picker.click()  # Close the date picker after fetching the days
    
    return available_days
    
def main():
    # Initialize undetected Chrome driver
    options = uc.ChromeOptions()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    # Initialize the WebDriver
    driver = uc.Chrome(options=options)

    # Open the website and wait for manual login
    driver.get("https://unabated.com/nba/odds")
    input("Please log in manually and navigate to the odds page, then press Enter to continue...")

    time.sleep(3)  # Adjust sleep time as needed

    # Get the total days to scan
    total_days_to_scan = 10000
    scanned_days = 0
    
    #pick sport
    sport = "nba"
    
    #ml,spread,total
    bet_type = "spread"
    
    #h1,q4
    quarter_type = "q4"
    
    #while scanned_days < total_days_to_scan:
    while True:
        print(f"Scanning for available days...")

        # Re-fetch the date picker each time to ensure it's fresh
        date_picker_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input.datepicker.form-control.datepicker-container'))
        )
        current_date = date_picker_input.get_attribute('value')
        current_month = current_date.split('/')[0]
        date = current_date.replace("/", "-")
        print(f"Current month: {current_month}")

        available_days = get_available_days(driver)
        
        for day in available_days:
            if scanned_days >= total_days_to_scan:
                break

            try:
                print(f"Processing day: {day}")
                change_date(driver, day)

                current_date = driver.find_element(By.CSS_SELECTOR, 'input.datepicker.form-control.datepicker-container').get_attribute('value')
                date = current_date.replace("/", "-")
                
                # Define the path for the merged CSV file
                merged_csv_path = f'unabated-database/{sport}/{quarter_type}/{bet_type}/merged/merged_{sport}_{quarter_type}_{bet_type}_{date}.csv'

                # Check if the file already exists and its size is greater than 100 KB
                if os.path.exists(merged_csv_path) and os.path.getsize(merged_csv_path) > 100 * 1024:
                    print(f"Skipping day {day} because {merged_csv_path} is already processed and larger than 100 KB.")
                    continue
                
                # Scrape the homepage data and save to CSV
                scrape_homepage(driver,sport,quarter_type,bet_type,date)
                scrape_live(driver,sport,quarter_type,bet_type,date)

                merge_info(live_path=f'unabated-database/{sport}/{quarter_type}/{bet_type}/live/live_{sport}_{quarter_type}_{bet_type}_{date}.csv', \
                            pregame_path=f'unabated-database/{sport}/{quarter_type}/{bet_type}/pregame/pregame_{sport}_{quarter_type}_{bet_type}_{date}.csv', \
                            output_path=f'unabated-database/{sport}/{quarter_type}/{bet_type}/merged/merged_{sport}_{quarter_type}_{bet_type}_{date}.csv')
                
                

                scanned_days += 1
                
            except StaleElementReferenceException as e:
                print(f"StaleElementReferenceException encountered. Retrying...")
                time.sleep(1)  # Add a slight delay before retrying

            

        if scanned_days < total_days_to_scan:
            go_to_next_month(driver)
        
    driver.quit()

if __name__ == "__main__":
    main()


Scanning for available days...
Current month: 4
Found 30 available days to scan: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
Processing day: 1
Attempting to change date to: 1
Successfully changed date to: 1
Skipping day 1 because unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-01-22.csv is already processed and larger than 100 KB.
Processing day: 2
Attempting to change date to: 2
Successfully changed date to: 2
Skipping day 2 because unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-02-22.csv is already processed and larger than 100 KB.
Processing day: 3
Attempting to change date to: 3
Successfully changed date to: 3
Skipping day 3 because unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-03-22.csv is already processed and larger than 100 KB.
Processing day: 4
Attempting to change date to: 4
Successfully changed date to: 4

C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pregame_df = pregame_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  live_df = live_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)


Merged data saved to unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-05-22.csv
Processing day: 6
Attempting to change date to: 6
Successfully changed date to: 6
Skipping day 6 because unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-06-22.csv is already processed and larger than 100 KB.
Processing day: 7
Attempting to change date to: 7
Successfully changed date to: 7
Left container row parsing ended at index 7 with error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[row-index="7"]"}
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E28923+23283]
	(No symbol) [0x00DEE934]
	(No symbol) [0x00D20733]
	(No symbol) [0x00D6326F]
	(No symbol) [0x00D634AB]
	(No symbol) [0x00D59571]
	(No symbol) [0x00D84464]
	(No symbol) [0x00D59485]
	(No symbol) [0x00D

C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pregame_df = pregame_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  live_df = live_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)


Merged data saved to unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-07-22.csv
Processing day: 8
Attempting to change date to: 8
Successfully changed date to: 8
Left container row parsing ended at index 9 with error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[row-index="9"]"}
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E28923+23283]
	(No symbol) [0x00DEE934]
	(No symbol) [0x00D20733]
	(No symbol) [0x00D6326F]
	(No symbol) [0x00D634AB]
	(No symbol) [0x00D59571]
	(No symbol) [0x00D84464]
	(No symbol) [0x00D59485]
	(No symbol) [0x00D846B4]
	(No symbol) [0x00D9CB8D]
	(No symbol) [0x00D841B6]
	(No symbol) [0x00D58017]
	(No symbol) [0x00D5890D]
	GetHandleVerifier [0x00F1A5E3+1013683]
	GetHandleVerifier [0x00F23E3C+1052684]
	GetHandleVerifier [0x00F1D4A4+

C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pregame_df = pregame_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  live_df = live_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)


Merged data saved to unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-08-22.csv
Processing day: 9
Attempting to change date to: 9
Successfully changed date to: 9
Left container row parsing ended at index 4 with error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[row-index="4"]"}
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E28923+23283]
	(No symbol) [0x00DEE934]
	(No symbol) [0x00D20733]
	(No symbol) [0x00D6326F]
	(No symbol) [0x00D634AB]
	(No symbol) [0x00D59571]
	(No symbol) [0x00D84464]
	(No symbol) [0x00D59485]
	(No symbol) [0x00D846B4]
	(No symbol) [0x00D9CB8D]
	(No symbol) [0x00D841B6]
	(No symbol) [0x00D58017]
	(No symbol) [0x00D5890D]
	GetHandleVerifier [0x00F1A5E3+1013683]
	GetHandleVerifier [0x00F23E3C+1052684]
	GetHandleVerifier [0x00F1D4A4+

C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pregame_df = pregame_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
C:\Users\justi\AppData\Local\Temp\ipykernel_21864\2719372656.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  live_df = live_df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)


Merged data saved to unabated-database/nba/q4/spread/merged/merged_nba_q4_spread_4-09-22.csv
Processing day: 10
Attempting to change date to: 10
Successfully changed date to: 10
Left container row parsing ended at index 0 with error: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00E28923+23283]
	(No symbol) [0x00DEE934]
	(No symbol) [0x00D20733]
	(No symbol) [0x00D260DF]
	(No symbol) [0x00D27D0F]
	(No symbol) [0x00D9D74B]
	(No symbol) [0x00D8441C]
	(No symbol) [0x00D9CB8D]
	(No symbol) [0x00D841B6]
	(No symbol) [0x00D58017]
	(No symbol) [0x00D5890D]
	GetHandleVerifier [0x00F1A5E3+1013683]
	GetHandleVerifier [0x00F23E3C+1052684]
	GetHandleVerifier [0x00F1D4A4+1025652]
	GetHandleVerifier [0x00E4EA2B+179195]
	(No symbol) [0x00DF6833

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.100)
Stacktrace:
	GetHandleVerifier [0x00E28923+23283]
	(No symbol) [0x00DEE934]
	(No symbol) [0x00D20733]
	(No symbol) [0x00CFD2E3]
	(No symbol) [0x00D8A64F]
	(No symbol) [0x00D9C686]
	(No symbol) [0x00D841B6]
	(No symbol) [0x00D58017]
	(No symbol) [0x00D5890D]
	GetHandleVerifier [0x00F1A5E3+1013683]
	GetHandleVerifier [0x00F23E3C+1052684]
	GetHandleVerifier [0x00F1D4A4+1025652]
	GetHandleVerifier [0x00E4EA2B+179195]
	(No symbol) [0x00DF6833]
	(No symbol) [0x00DF3198]
	(No symbol) [0x00DF3337]
	(No symbol) [0x00DEB4BE]
	BaseThreadInitThunk [0x75DC7BA9+25]
	RtlInitializeExceptionChain [0x77D9C10B+107]
	RtlClearBits [0x77D9C08F+191]
